# Deep Dive 02: DragonNet 多处理扩展

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **深入理解** DragonNet 的网络架构和 Loss 设计原理
2. **掌握** 如何将二元处理扩展到多处理（Multi-Treatment）场景
3. **实现** 完整的 Multi-Treatment DragonNet（从零实现，不调包）
4. **应用** 到真实业务场景：多种优惠券策略选择

---

## 📖 背景故事

> **场景：电商平台优惠券策略选择**
>
> 你是某电商平台的算法工程师。平台有多种优惠券类型：
> - **无券 (T=0)**：不发券
> - **5折券 (T=1)**：商品5折，成本高但吸引力大
> - **7折券 (T=2)**：商品7折，成本中等
> - **满100减20 (T=3)**：门槛券，提高客单价
>
> 业务问题：**对于每个用户，应该发哪种券？**
>
> 你决定用 DragonNet 来估计每种券对每个用户的效果（CATE），但发现原版 DragonNet 只支持二元处理...
>
> **本 notebook 就是来解决这个问题的！**

---

## Part 1: 原始 DragonNet 架构回顾

在扩展之前，我们先彻底理解原始 DragonNet 的设计。

### 1.1 DragonNet 核心思想

DragonNet (Shi et al., 2019) 的核心创新是**用神经网络同时学习**：
1. **表示学习 (Representation)**: 学习协变量 X 的表示 Z = φ(X)
2. **倾向得分预测 (Propensity Score)**: 从表示预测处理概率 P(T=1|X)
3. **结果预测 (Outcome)**: 从表示预测潜在结果 Y(0), Y(1)

**关键洞见**：倾向得分预测任务帮助表示层学习到"处理分配相关"的特征，这些特征正是混淆变量的体现！

### 1.2 原始 DragonNet 网络结构

```
        输入 X
           │
           ▼
   ┌───────────────┐
   │   共享表示层   │  φ(X) → Z
   │ (Representation)│
   └───────┬───────┘
           │
     ┌─────┴─────┐
     │           │
     ▼           ▼
┌─────────┐ ┌─────────┐
│ 倾向得分 │ │ 结果预测 │
│  头 (ε)  │ │   头    │
└────┬────┘ └────┬────┘
     │           │
     ▼           ▼
  P(T=1|X)    ┌──┴──┐
              │     │
              ▼     ▼
           μ₀(X)  μ₁(X)
```

### 1.3 原始 DragonNet Loss 函数

DragonNet 的 Loss 包含三部分：

$$\mathcal{L} = \mathcal{L}_{outcome} + \alpha \cdot \mathcal{L}_{propensity} + \beta \cdot \mathcal{L}_{targeted}$$

其中：

1. **Outcome Loss** (观测结果拟合):
$$\mathcal{L}_{outcome} = \frac{1}{n}\sum_{i} \left[ T_i (Y_i - \hat{\mu}_1(X_i))^2 + (1-T_i)(Y_i - \hat{\mu}_0(X_i))^2 \right]$$

2. **Propensity Loss** (倾向得分拟合):
$$\mathcal{L}_{propensity} = -\frac{1}{n}\sum_{i} \left[ T_i \log \hat{\epsilon}(X_i) + (1-T_i) \log(1-\hat{\epsilon}(X_i)) \right]$$

3. **Targeted Regularization** (Neyman 正交化):
$$\mathcal{L}_{targeted} = \frac{1}{n}\sum_{i} \left( \hat{Y}_i - Y_i + \frac{T_i - \hat{\epsilon}(X_i)}{\hat{\epsilon}(X_i)(1-\hat{\epsilon}(X_i))} (Y_i - \hat{Y}_i) \right)^2$$

其中 $\hat{Y}_i = T_i \hat{\mu}_1(X_i) + (1-T_i) \hat{\mu}_0(X_i)$

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")
print(f"PyTorch 版本: {torch.__version__}")

### 1.4 原始 DragonNet 实现（供参考）

In [ ]:
class OriginalDragonNet(nn.Module):
    """
    原始 DragonNet - 仅支持二元处理
    用于对比和理解
    """
    def __init__(self, input_dim, hidden_dims=[200, 100, 100]):
        super().__init__()
        
        # 共享表示层
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.ELU(),
            ])
            prev_dim = h_dim
        self.representation = nn.Sequential(*layers)
        
        # 倾向得分头（二分类）
        self.propensity_head = nn.Sequential(
            nn.Linear(hidden_dims[-1], 1),
            nn.Sigmoid()
        )
        
        # 结果预测头（两个：μ₀ 和 μ₁）
        self.mu0_head = nn.Sequential(
            nn.Linear(hidden_dims[-1], 100),
            nn.ELU(),
            nn.Linear(100, 1)
        )
        self.mu1_head = nn.Sequential(
            nn.Linear(hidden_dims[-1], 100),
            nn.ELU(),
            nn.Linear(100, 1)
        )
    
    def forward(self, x):
        # 共享表示
        z = self.representation(x)
        
        # 倾向得分
        propensity = self.propensity_head(z).squeeze(-1)  # P(T=1|X)
        
        # 潜在结果
        mu0 = self.mu0_head(z).squeeze(-1)  # E[Y(0)|X]
        mu1 = self.mu1_head(z).squeeze(-1)  # E[Y(1)|X]
        
        return propensity, mu0, mu1
    
    def predict_cate(self, x):
        """预测 CATE = E[Y(1) - Y(0)|X]"""
        _, mu0, mu1 = self.forward(x)
        return mu1 - mu0

print("✅ 原始 DragonNet 定义完成")

---

## Part 2: 多处理扩展的挑战

### 2.1 为什么不能直接用原版 DragonNet？

原版 DragonNet 的限制：

| 组件 | 原版设计 | 多处理需求 |
|------|---------|----------|
| 倾向得分 | 二分类 Sigmoid | 多分类 Softmax |
| 结果预测头 | 2 个 (μ₀, μ₁) | K 个 (μ₀, ..., μₖ₋₁) |
| Outcome Loss | 二选一 | K选一 |
| Targeted Reg | 针对二元处理设计 | 需要泛化 |

### 2.2 设计决策点

在扩展 DragonNet 时，我们需要做出以下设计决策：

1. **处理编码方式**：One-hot vs Embedding
2. **结果预测头设计**：独立头 vs 共享底层
3. **Targeted Regularization 泛化**：如何处理多个处理的正交化

---

## Part 3: Multi-Treatment DragonNet 架构设计

### 3.1 网络架构

```
           输入 X
              │
              ▼
      ┌───────────────┐
      │   共享表示层   │  φ(X) → Z
      │ (Representation)│
      └───────┬───────┘
              │
        ┌─────┴─────┐
        │           │
        ▼           ▼
   ┌─────────┐  ┌─────────────────────┐
   │ 倾向得分 │  │    结果预测头        │
   │  多分类  │  │ (每个处理一个头)    │
   └────┬────┘  └────┬────┬────┬────┘
        │            │    │    │    │
        ▼            ▼    ▼    ▼    ▼
   P(T=k|X)       μ₀(X) μ₁(X) μ₂(X) μ₃(X)
   k=0,1,2,3
```

### 3.2 Loss 函数设计

#### 3.2.1 多处理 Outcome Loss

$$\mathcal{L}_{outcome} = \frac{1}{n}\sum_{i} \sum_{k=0}^{K-1} \mathbb{1}[T_i = k] (Y_i - \hat{\mu}_k(X_i))^2$$

即：只对观测到的处理计算 MSE。

#### 3.2.2 多分类 Propensity Loss

$$\mathcal{L}_{propensity} = -\frac{1}{n}\sum_{i} \sum_{k=0}^{K-1} \mathbb{1}[T_i = k] \log \hat{\pi}_k(X_i)$$

即：标准的多分类交叉熵。

#### 3.2.3 泛化的 Targeted Regularization

对于多处理，我们需要泛化 Targeted Regularization。核心思想是使用**广义倾向得分加权**：

$$\mathcal{L}_{targeted} = \frac{1}{n}\sum_{i} \left( \hat{Y}_i - Y_i + \frac{\mathbb{1}[T_i = k] - \hat{\pi}_k(X_i)}{\hat{\pi}_k(X_i)} (Y_i - \hat{\mu}_k(X_i)) \right)^2$$

其中 $\hat{Y}_i = \sum_{k} \mathbb{1}[T_i=k] \hat{\mu}_k(X_i)$

---

## Part 4: 从零实现 Multi-Treatment DragonNet

### 🧪 TODO 练习 1: 实现 Multi-Treatment DragonNet

下面是 Multi-Treatment DragonNet 的框架，请完成 TODO 部分。

**提示**：
1. 倾向得分头：使用 `nn.Linear(hidden_dim, num_treatments)` + `F.softmax`
2. 结果预测头：使用 `nn.ModuleList` 存储 K 个独立的头
3. 前向传播：返回 propensity scores 和所有 mu 预测

In [ ]:
class MultiTreatmentDragonNet(nn.Module):
    """
    多处理 DragonNet
    支持 K 种处理（包括不处理）
    """
    def __init__(self, input_dim, num_treatments, hidden_dims=[200, 100, 100], outcome_hidden=100):
        super().__init__()
        
        self.num_treatments = num_treatments
        
        # 实现共享表示层
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.ELU()
            ])
            prev_dim = h_dim
        self.representation = nn.Sequential(*layers)
        
        # 实现倾向得分头（多分类）
        self.propensity_head = nn.Linear(hidden_dims[-1], num_treatments)
        
        # 实现结果预测头（每个处理一个）
        self.outcome_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dims[-1], outcome_hidden),
                nn.ELU(),
                nn.Linear(outcome_hidden, 1)
            )
            for _ in range(num_treatments)
        ])
    
    def forward(self, x):
        """
        前向传播
        
        Returns:
            propensity: (batch_size, num_treatments) - 每种处理的概率
            mus: (batch_size, num_treatments) - 每种处理下的预测结果
        """
        # 实现前向传播
        # 1. 获取共享表示
        z = self.representation(x)
        
        # 2. 计算倾向得分
        propensity = F.softmax(self.propensity_head(z), dim=-1)
        
        # 3. 计算每个处理的预测结果
        mus = torch.cat([head(z) for head in self.outcome_heads], dim=-1)
        
        return propensity, mus
    
    def predict_cate(self, x, reference_treatment=0):
        """
        预测 CATE
        CATE_k = E[Y(k) - Y(reference)|X]
        
        Args:
            x: 输入特征
            reference_treatment: 参照组（通常是不处理，即 T=0）
            
        Returns:
            cate: (batch_size, num_treatments - 1) - 每种处理相对参照组的 CATE
        """
        _, mus = self.forward(x)
        mu_ref = mus[:, reference_treatment:reference_treatment+1]
        # 计算相对于参照组的 CATE
        cate = mus - mu_ref
        # 移除参照组自身（CATE=0）
        mask = torch.ones(self.num_treatments, dtype=torch.bool)
        mask[reference_treatment] = False
        return cate[:, mask]

print("✅ MultiTreatmentDragonNet 类定义完成（已完成 TODO 部分）")

### 📝 参考答案

In [ ]:
# ============================================================
# 参考答案：Multi-Treatment DragonNet 完整实现
# ============================================================

class MultiTreatmentDragonNet(nn.Module):
    """
    多处理 DragonNet - 完整实现
    """
    def __init__(self, input_dim, num_treatments, hidden_dims=[200, 100, 100], outcome_hidden=100):
        super().__init__()
        
        self.num_treatments = num_treatments
        
        # 共享表示层
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.ELU(),
            ])
            prev_dim = h_dim
        self.representation = nn.Sequential(*layers)
        
        # 倾向得分头（多分类）
        self.propensity_head = nn.Linear(hidden_dims[-1], num_treatments)
        
        # 结果预测头（每个处理一个）
        self.outcome_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dims[-1], outcome_hidden),
                nn.ELU(),
                nn.Linear(outcome_hidden, 1)
            )
            for _ in range(num_treatments)
        ])
    
    def forward(self, x):
        # 共享表示
        z = self.representation(x)
        
        # 倾向得分（多分类）
        propensity = F.softmax(self.propensity_head(z), dim=-1)
        
        # 每个处理的预测结果
        mus = torch.cat([head(z) for head in self.outcome_heads], dim=-1)
        
        return propensity, mus
    
    def predict_cate(self, x, reference_treatment=0):
        """预测相对于参照组的 CATE"""
        _, mus = self.forward(x)
        mu_ref = mus[:, reference_treatment:reference_treatment+1]
        cate = mus - mu_ref
        mask = torch.ones(self.num_treatments, dtype=torch.bool)
        mask[reference_treatment] = False
        return cate[:, mask]

# 测试
model = MultiTreatmentDragonNet(input_dim=10, num_treatments=4)
x_test = torch.randn(5, 10)
prop, mus = model(x_test)
print(f"✅ 模型创建成功")
print(f"   输入维度: 10, 处理数量: 4")
print(f"   倾向得分形状: {prop.shape}  # (batch_size, num_treatments)")
print(f"   预测结果形状: {mus.shape}  # (batch_size, num_treatments)")
print(f"   倾向得分和: {prop.sum(dim=1)}  # 应该全是 1")

---

## Part 5: 实现 Multi-Treatment Loss 函数

### 🧪 TODO 练习 2: 实现多处理 Loss

请实现完整的 Multi-Treatment DragonNet Loss，包括：
1. Outcome Loss（只对观测到的处理计算）
2. Propensity Loss（多分类交叉熵）
3. Targeted Regularization（泛化版本）

In [ ]:
class MultiTreatmentDragonNetLoss(nn.Module):
    """
    多处理 DragonNet Loss 函数
    """
    def __init__(self, alpha=1.0, beta=1.0, eps=1e-6):
        super().__init__()
        self.alpha = alpha  # 倾向得分 Loss 权重
        self.beta = beta    # Targeted Reg 权重
        self.eps = eps      # 数值稳定性
    
    def forward(self, propensity, mus, treatment, outcome):
        """
        Args:
            propensity: (batch_size, num_treatments) - 预测的倾向得分
            mus: (batch_size, num_treatments) - 预测的潜在结果
            treatment: (batch_size,) - 实际处理（整数 0, 1, ..., K-1）
            outcome: (batch_size,) - 实际结果
        """
        batch_size = treatment.shape[0]
        num_treatments = propensity.shape[1]
        
        # 实现 Outcome Loss
        # 使用 gather 选择对应处理的预测值
        mu_observed = mus.gather(1, treatment.unsqueeze(1)).squeeze(1)
        outcome_loss = F.mse_loss(mu_observed, outcome)
        
        # 实现 Propensity Loss
        # 使用 F.cross_entropy
        propensity_loss = F.cross_entropy(propensity + self.eps, treatment)
        
        # 实现 Targeted Regularization
        # 1. 获取观测处理的倾向得分 pi_t
        pi_observed = propensity.gather(1, treatment.unsqueeze(1)).squeeze(1)
        pi_observed = torch.clamp(pi_observed, min=self.eps)  # 数值稳定
        
        # 2. 计算 correction term
        correction = (1.0 - pi_observed) / pi_observed * (outcome - mu_observed)
        
        # 3. Targeted regularization term
        targeted_term = mu_observed - outcome + correction
        targeted_loss = (targeted_term ** 2).mean()
        
        total_loss = outcome_loss + self.alpha * propensity_loss + self.beta * targeted_loss
        
        return total_loss, {
            'outcome_loss': outcome_loss.item(),
            'propensity_loss': propensity_loss.item(),
            'targeted_loss': targeted_loss.item(),
            'total_loss': total_loss.item()
        }

print("✅ MultiTreatmentDragonNetLoss 类定义完成（已完成 TODO 部分）")

### 📝 参考答案

In [ ]:
# ============================================================
# 参考答案：Multi-Treatment DragonNet Loss 完整实现
# ============================================================

class MultiTreatmentDragonNetLoss(nn.Module):
    """
    多处理 DragonNet Loss 函数 - 完整实现
    """
    def __init__(self, alpha=1.0, beta=1.0, eps=1e-6):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.eps = eps  # 数值稳定性
    
    def forward(self, propensity, mus, treatment, outcome):
        batch_size = treatment.shape[0]
        num_treatments = propensity.shape[1]
        
        # 1. Outcome Loss: 只对观测到的处理计算 MSE
        mu_observed = mus.gather(1, treatment.unsqueeze(1)).squeeze(1)
        outcome_loss = F.mse_loss(mu_observed, outcome)
        
        # 2. Propensity Loss: 多分类交叉熵
        propensity_loss = F.cross_entropy(propensity + self.eps, treatment)
        
        # 3. Targeted Regularization (泛化版)
        # 获取观测处理的倾向得分
        pi_observed = propensity.gather(1, treatment.unsqueeze(1)).squeeze(1)
        pi_observed = torch.clamp(pi_observed, min=self.eps)  # 数值稳定
        
        # 构建指示变量 (1[T=k] = 1 对于观测到的处理)
        # 计算 correction term
        correction = (1.0 - pi_observed) / pi_observed * (outcome - mu_observed)
        
        # Targeted regularization term
        targeted_term = mu_observed - outcome + correction
        targeted_loss = (targeted_term ** 2).mean()
        
        # 总 Loss
        total_loss = outcome_loss + self.alpha * propensity_loss + self.beta * targeted_loss
        
        return total_loss, {
            'outcome_loss': outcome_loss.item(),
            'propensity_loss': propensity_loss.item(),
            'targeted_loss': targeted_loss.item(),
            'total_loss': total_loss.item()
        }

# 测试 Loss 函数
loss_fn = MultiTreatmentDragonNetLoss()
propensity = torch.softmax(torch.randn(5, 4), dim=-1)
mus = torch.randn(5, 4)
treatment = torch.randint(0, 4, (5,))
outcome = torch.randn(5)

loss, loss_dict = loss_fn(propensity, mus, treatment, outcome)
print(f"✅ Loss 函数测试成功")
print(f"   Outcome Loss: {loss_dict['outcome_loss']:.4f}")
print(f"   Propensity Loss: {loss_dict['propensity_loss']:.4f}")
print(f"   Targeted Loss: {loss_dict['targeted_loss']:.4f}")
print(f"   Total Loss: {loss_dict['total_loss']:.4f}")

---

## Part 6: 生成多处理模拟数据

为了测试我们的模型，我们需要生成一个真实的多处理场景数据集。

In [ ]:
def generate_multi_treatment_data(n_samples=5000, seed=42):
    """
    生成多处理模拟数据
    
    场景：电商优惠券
    - T=0: 不发券
    - T=1: 5折券（高成本，高效果）
    - T=2: 7折券（中成本，中效果）
    - T=3: 满100减20（低成本，效果依赖用户类型）
    
    用户特征：
    - X0: 购买力（高购买力用户更容易被发券）
    - X1: 价格敏感度（高敏感度用户对折扣响应更大）
    - X2: 活跃度（影响基础转化率）
    - X3-X9: 其他特征
    """
    np.random.seed(seed)
    
    # 用户特征
    X = np.random.randn(n_samples, 10)
    
    # 关键特征
    purchase_power = X[:, 0]      # 购买力
    price_sensitivity = X[:, 1]   # 价格敏感度
    activity = X[:, 2]            # 活跃度
    
    # 真实的倾向得分（处理分配机制）
    # 高购买力用户更容易被发大额券
    logits = np.zeros((n_samples, 4))
    logits[:, 0] = 0  # 基准：不发券
    logits[:, 1] = -1.0 + 1.5 * purchase_power + 0.3 * activity  # 5折券：给高价值用户
    logits[:, 2] = 0.5 + 0.5 * purchase_power + 0.5 * price_sensitivity  # 7折券：较普遍
    logits[:, 3] = 0.3 + 0.3 * purchase_power + 0.8 * activity  # 满减券：给活跃用户
    
    # Softmax 得到概率
    exp_logits = np.exp(logits - logits.max(axis=1, keepdims=True))
    true_propensity = exp_logits / exp_logits.sum(axis=1, keepdims=True)
    
    # 根据概率分配处理
    treatment = np.array([np.random.choice(4, p=p) for p in true_propensity])
    
    # 真实的潜在结果函数
    def mu_0(X):  # 不发券
        return 10 + 3 * X[:, 2] + np.random.randn(len(X)) * 0.5
    
    def mu_1(X):  # 5折券 - 对价格敏感用户效果更大
        base = mu_0(X)
        treatment_effect = 8 + 4 * X[:, 1]  # 价格敏感度越高，效果越大
        return base + treatment_effect
    
    def mu_2(X):  # 7折券 - 中等效果
        base = mu_0(X)
        treatment_effect = 5 + 2 * X[:, 1]
        return base + treatment_effect
    
    def mu_3(X):  # 满减券 - 对活跃用户效果更大
        base = mu_0(X)
        treatment_effect = 3 + 3 * X[:, 2]  # 活跃度越高，效果越大
        return base + treatment_effect
    
    # 计算所有潜在结果（用于评估真实 CATE）
    mu_all = np.stack([mu_0(X), mu_1(X), mu_2(X), mu_3(X)], axis=1)
    
    # 观测结果
    outcome = np.array([mu_all[i, t] for i, t in enumerate(treatment)])
    
    # 真实 CATE（相对于不发券）
    true_cate = mu_all[:, 1:] - mu_all[:, 0:1]
    
    return {
        'X': X,
        'treatment': treatment,
        'outcome': outcome,
        'true_cate': true_cate,  # (n_samples, 3)
        'true_mu': mu_all,
        'true_propensity': true_propensity
    }

# 生成数据
data = generate_multi_treatment_data(n_samples=8000)
print(f"✅ 数据生成完成")
print(f"   样本数: {len(data['X'])}")
print(f"   特征维度: {data['X'].shape[1]}")
print(f"   处理分布: {np.bincount(data['treatment'])}")
print(f"   真实 CATE 均值 (T=1,2,3 相对 T=0):")
print(f"      5折券: {data['true_cate'][:, 0].mean():.2f}")
print(f"      7折券: {data['true_cate'][:, 1].mean():.2f}")
print(f"      满减券: {data['true_cate'][:, 2].mean():.2f}")

In [ ]:
# 可视化数据分布
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 处理分布
ax1 = axes[0, 0]
treatment_labels = ['不发券', '5折券', '7折券', '满减券']
counts = np.bincount(data['treatment'])
ax1.bar(treatment_labels, counts, color=['#95a5a6', '#e74c3c', '#3498db', '#2ecc71'])
ax1.set_ylabel('样本数')
ax1.set_title('处理分布')

# 结果分布（按处理）
ax2 = axes[0, 1]
for t in range(4):
    mask = data['treatment'] == t
    ax2.hist(data['outcome'][mask], bins=30, alpha=0.5, label=treatment_labels[t])
ax2.set_xlabel('结果 (GMV)')
ax2.set_ylabel('频数')
ax2.set_title('各处理组结果分布')
ax2.legend()

# 真实 CATE 分布
ax3 = axes[1, 0]
for i, label in enumerate(['5折券', '7折券', '满减券']):
    ax3.hist(data['true_cate'][:, i], bins=30, alpha=0.5, label=label)
ax3.axvline(x=0, color='black', linestyle='--', alpha=0.5)
ax3.set_xlabel('CATE (相对不发券)')
ax3.set_ylabel('频数')
ax3.set_title('真实 CATE 分布（异质性效应）')
ax3.legend()

# 特征与处理的关系
ax4 = axes[1, 1]
for t in range(4):
    mask = data['treatment'] == t
    ax4.scatter(data['X'][mask, 0], data['X'][mask, 1], 
               alpha=0.3, label=treatment_labels[t], s=10)
ax4.set_xlabel('购买力 (X0)')
ax4.set_ylabel('价格敏感度 (X1)')
ax4.set_title('处理分配与特征关系（存在混淆）')
ax4.legend()

plt.tight_layout()
plt.show()

print("\n观察：")
print("1. 处理分布不均匀（模拟真实业务场景）")
print("2. 各处理组结果分布有重叠但不完全相同")
print("3. CATE 存在异质性（不同用户效果不同）")
print("4. 处理分配与用户特征相关（存在混淆偏差）")

---

## Part 7: 训练 Multi-Treatment DragonNet

In [ ]:
def train_multi_treatment_dragonnet(data, epochs=100, batch_size=256, lr=1e-3, 
                                     alpha=1.0, beta=1.0, verbose=True):
    """
    训练 Multi-Treatment DragonNet
    """
    # 数据准备
    X_train, X_val, t_train, t_val, y_train, y_val = train_test_split(
        data['X'], data['treatment'], data['outcome'], 
        test_size=0.2, random_state=42
    )
    
    # 标准化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    # 转为 Tensor
    X_train_t = torch.FloatTensor(X_train)
    t_train_t = torch.LongTensor(t_train)
    y_train_t = torch.FloatTensor(y_train)
    X_val_t = torch.FloatTensor(X_val)
    t_val_t = torch.LongTensor(t_val)
    y_val_t = torch.FloatTensor(y_val)
    
    # 创建 DataLoader
    train_dataset = TensorDataset(X_train_t, t_train_t, y_train_t)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # 初始化模型
    model = MultiTreatmentDragonNet(
        input_dim=X_train.shape[1],
        num_treatments=4,
        hidden_dims=[200, 100, 100]
    )
    
    loss_fn = MultiTreatmentDragonNetLoss(alpha=alpha, beta=beta)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    
    # 训练历史
    history = {
        'train_loss': [], 'val_loss': [],
        'outcome_loss': [], 'propensity_loss': [], 'targeted_loss': []
    }
    
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(epochs):
        # 训练
        model.train()
        train_losses = []
        epoch_metrics = {'outcome_loss': [], 'propensity_loss': [], 'targeted_loss': []}
        
        for X_batch, t_batch, y_batch in train_loader:
            optimizer.zero_grad()
            propensity, mus = model(X_batch)
            loss, metrics = loss_fn(propensity, mus, t_batch, y_batch)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
            for k, v in metrics.items():
                if k in epoch_metrics:
                    epoch_metrics[k].append(v)
        
        # 验证
        model.eval()
        with torch.no_grad():
            propensity_val, mus_val = model(X_val_t)
            val_loss, _ = loss_fn(propensity_val, mus_val, t_val_t, y_val_t)
        
        # 记录历史
        avg_train_loss = np.mean(train_losses)
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(val_loss.item())
        for k in epoch_metrics:
            history[k].append(np.mean(epoch_metrics[k]))
        
        # 学习率调度
        scheduler.step(val_loss)
        
        # 保存最佳模型
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_model_state = model.state_dict().copy()
        
        if verbose and (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs} | "
                  f"Train Loss: {avg_train_loss:.4f} | "
                  f"Val Loss: {val_loss.item():.4f}")
    
    # 加载最佳模型
    model.load_state_dict(best_model_state)
    
    return model, scaler, history

# 训练模型
print("开始训练 Multi-Treatment DragonNet...\n")
model, scaler, history = train_multi_treatment_dragonnet(
    data, epochs=150, batch_size=256, lr=5e-4,
    alpha=1.0, beta=1.0
)
print("\n✅ 训练完成")

In [ ]:
# 可视化训练过程
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 总 Loss
ax1 = axes[0]
ax1.plot(history['train_loss'], label='Train', alpha=0.8)
ax1.plot(history['val_loss'], label='Validation', alpha=0.8)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('训练过程')
ax1.legend()
ax1.set_yscale('log')

# 各部分 Loss
ax2 = axes[1]
ax2.plot(history['outcome_loss'], label='Outcome Loss', alpha=0.8)
ax2.plot(history['propensity_loss'], label='Propensity Loss', alpha=0.8)
ax2.plot(history['targeted_loss'], label='Targeted Reg', alpha=0.8)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('各部分 Loss')
ax2.legend()
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

---

## Part 8: 评估模型性能

### 8.1 CATE 预测评估

In [ ]:
def evaluate_cate_prediction(model, scaler, data):
    """
    评估 CATE 预测性能
    """
    model.eval()
    X_scaled = torch.FloatTensor(scaler.transform(data['X']))
    
    with torch.no_grad():
        # 预测 CATE
        pred_cate = model.predict_cate(X_scaled, reference_treatment=0)
        pred_cate = pred_cate.numpy()
    
    true_cate = data['true_cate']
    
    results = {}
    treatment_names = ['5折券', '7折券', '满减券']
    
    print("=" * 60)
    print("CATE 预测评估结果")
    print("=" * 60)
    
    for i, name in enumerate(treatment_names):
        # RMSE
        rmse = np.sqrt(np.mean((pred_cate[:, i] - true_cate[:, i]) ** 2))
        # MAE
        mae = np.mean(np.abs(pred_cate[:, i] - true_cate[:, i]))
        # 相关系数
        corr = np.corrcoef(pred_cate[:, i], true_cate[:, i])[0, 1]
        # 均值偏差
        bias = np.mean(pred_cate[:, i]) - np.mean(true_cate[:, i])
        
        results[name] = {'rmse': rmse, 'mae': mae, 'corr': corr, 'bias': bias}
        
        print(f"\n{name}:")
        print(f"  RMSE: {rmse:.3f}")
        print(f"  MAE:  {mae:.3f}")
        print(f"  相关系数: {corr:.3f}")
        print(f"  均值偏差: {bias:.3f} (真实均值: {np.mean(true_cate[:, i]):.3f})")
    
    return pred_cate, results

pred_cate, eval_results = evaluate_cate_prediction(model, scaler, data)

In [ ]:
# 可视化 CATE 预测 vs 真实
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
treatment_names = ['5折券', '7折券', '满减券']
colors = ['#e74c3c', '#3498db', '#2ecc71']

for i, (name, color) in enumerate(zip(treatment_names, colors)):
    ax = axes[i]
    
    # 散点图
    ax.scatter(data['true_cate'][:, i], pred_cate[:, i], 
              alpha=0.3, s=10, c=color)
    
    # 对角线
    min_val = min(data['true_cate'][:, i].min(), pred_cate[:, i].min())
    max_val = max(data['true_cate'][:, i].max(), pred_cate[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='完美预测')
    
    ax.set_xlabel('真实 CATE')
    ax.set_ylabel('预测 CATE')
    ax.set_title(f'{name}\nCorr: {eval_results[name]["corr"]:.3f}')
    ax.legend()

plt.suptitle('CATE 预测 vs 真实', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 8.2 最优处理选择评估

In [ ]:
def evaluate_optimal_treatment_selection(model, scaler, data):
    """
    评估最优处理选择性能
    
    关键指标：
    - 最优处理选择准确率
    - Value 函数提升（使用模型选择 vs 随机 vs 全发）
    """
    model.eval()
    X_scaled = torch.FloatTensor(scaler.transform(data['X']))
    
    with torch.no_grad():
        _, pred_mus = model(X_scaled)
        pred_mus = pred_mus.numpy()
    
    # 真实潜在结果
    true_mus = data['true_mu']
    
    # 模型推荐的最优处理
    pred_optimal = np.argmax(pred_mus, axis=1)
    
    # 真实最优处理
    true_optimal = np.argmax(true_mus, axis=1)
    
    # 最优处理选择准确率
    accuracy = np.mean(pred_optimal == true_optimal)
    
    # Value 函数计算
    n_samples = len(true_mus)
    
    # 1. 使用模型推荐
    value_model = np.mean([true_mus[i, pred_optimal[i]] for i in range(n_samples)])
    
    # 2. 随机选择
    np.random.seed(42)
    random_treatment = np.random.randint(0, 4, n_samples)
    value_random = np.mean([true_mus[i, random_treatment[i]] for i in range(n_samples)])
    
    # 3. 全部不发券
    value_no_treatment = np.mean(true_mus[:, 0])
    
    # 4. 全部发5折券
    value_all_50off = np.mean(true_mus[:, 1])
    
    # 5. Oracle（真实最优）
    value_oracle = np.mean([true_mus[i, true_optimal[i]] for i in range(n_samples)])
    
    print("=" * 60)
    print("最优处理选择评估")
    print("=" * 60)
    print(f"\n最优处理选择准确率: {accuracy:.2%}")
    print(f"\nValue 函数对比 (期望结果):")
    print(f"  随机选择:      {value_random:.2f}")
    print(f"  全部不发券:    {value_no_treatment:.2f}")
    print(f"  全发5折券:     {value_all_50off:.2f}")
    print(f"  模型推荐:      {value_model:.2f}")
    print(f"  Oracle(最优): {value_oracle:.2f}")
    print(f"\n模型 vs 随机 提升: {(value_model - value_random) / value_random:.1%}")
    print(f"模型 vs Oracle 差距: {(value_oracle - value_model) / value_oracle:.1%}")
    
    return {
        'accuracy': accuracy,
        'value_model': value_model,
        'value_random': value_random,
        'value_no_treatment': value_no_treatment,
        'value_all_50off': value_all_50off,
        'value_oracle': value_oracle,
        'pred_optimal': pred_optimal,
        'true_optimal': true_optimal
    }

selection_results = evaluate_optimal_treatment_selection(model, scaler, data)

In [ ]:
# 可视化策略对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Value 函数对比
ax1 = axes[0]
strategies = ['随机', '不发券', '全发5折', '模型推荐', 'Oracle']
values = [
    selection_results['value_random'],
    selection_results['value_no_treatment'],
    selection_results['value_all_50off'],
    selection_results['value_model'],
    selection_results['value_oracle']
]
colors = ['#95a5a6', '#e74c3c', '#f39c12', '#2ecc71', '#9b59b6']
bars = ax1.bar(strategies, values, color=colors)
ax1.set_ylabel('期望结果 (GMV)')
ax1.set_title('不同策略的期望结果对比')
ax1.axhline(y=selection_results['value_model'], color='#2ecc71', linestyle='--', alpha=0.5)

# 添加数值标签
for bar, val in zip(bars, values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
            f'{val:.1f}', ha='center', va='bottom')

# 最优处理分布对比
ax2 = axes[1]
treatment_names = ['不发券', '5折券', '7折券', '满减券']
x = np.arange(4)
width = 0.35

true_dist = np.bincount(selection_results['true_optimal'], minlength=4) / len(selection_results['true_optimal'])
pred_dist = np.bincount(selection_results['pred_optimal'], minlength=4) / len(selection_results['pred_optimal'])

ax2.bar(x - width/2, true_dist, width, label='真实最优', color='#3498db', alpha=0.8)
ax2.bar(x + width/2, pred_dist, width, label='模型推荐', color='#e74c3c', alpha=0.8)
ax2.set_xticks(x)
ax2.set_xticklabels(treatment_names)
ax2.set_ylabel('比例')
ax2.set_title('最优处理分布对比')
ax2.legend()

plt.tight_layout()
plt.show()

---

## Part 9: 高级技巧：处理类别不平衡

在实际业务中，不同处理的样本量可能严重不平衡。下面介绍几种处理方法。

In [ ]:
class MultiTreatmentDragonNetLossWithBalancing(nn.Module):
    """
    带类别平衡的 Multi-Treatment DragonNet Loss
    
    改进点：
    1. 倾向得分 Loss 使用类别权重
    2. Outcome Loss 使用处理组权重
    """
    def __init__(self, alpha=1.0, beta=1.0, eps=1e-6, treatment_weights=None):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.eps = eps
        self.treatment_weights = treatment_weights  # 类别权重
    
    def forward(self, propensity, mus, treatment, outcome):
        batch_size = treatment.shape[0]
        
        # 1. Outcome Loss with balancing
        mu_observed = mus.gather(1, treatment.unsqueeze(1)).squeeze(1)
        
        if self.treatment_weights is not None:
            # 每个样本根据其处理组获得权重
            sample_weights = self.treatment_weights[treatment]
            outcome_loss = (sample_weights * (mu_observed - outcome) ** 2).mean()
        else:
            outcome_loss = F.mse_loss(mu_observed, outcome)
        
        # 2. Propensity Loss with class weights
        if self.treatment_weights is not None:
            propensity_loss = F.cross_entropy(
                propensity + self.eps, treatment,
                weight=self.treatment_weights
            )
        else:
            propensity_loss = F.cross_entropy(propensity + self.eps, treatment)
        
        # 3. Targeted Regularization
        pi_observed = propensity.gather(1, treatment.unsqueeze(1)).squeeze(1)
        pi_observed = torch.clamp(pi_observed, min=self.eps)
        correction = (1.0 - pi_observed) / pi_observed * (outcome - mu_observed)
        targeted_term = mu_observed - outcome + correction
        targeted_loss = (targeted_term ** 2).mean()
        
        total_loss = outcome_loss + self.alpha * propensity_loss + self.beta * targeted_loss
        
        return total_loss, {
            'outcome_loss': outcome_loss.item(),
            'propensity_loss': propensity_loss.item(),
            'targeted_loss': targeted_loss.item(),
            'total_loss': total_loss.item()
        }

# 计算类别权重（逆频率）
treatment_counts = np.bincount(data['treatment'])
treatment_weights = torch.FloatTensor(1.0 / treatment_counts)
treatment_weights = treatment_weights / treatment_weights.sum() * 4  # 归一化

print("处理组样本分布:")
for i, (count, weight) in enumerate(zip(treatment_counts, treatment_weights.numpy())):
    print(f"  T={i}: {count} 样本, 权重 = {weight:.3f}")

---

## Part 10: 与其他方法对比

### 🧪 TODO 练习 3: 实现 Multi-Treatment T-Learner 作为基线

为了对比，请实现一个简单的 Multi-Treatment T-Learner。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

def multi_treatment_t_learner(X, treatment, outcome, X_eval):
    """
    Multi-Treatment T-Learner
    
    为每种处理训练一个独立模型
    
    Args:
        X: 训练特征
        treatment: 处理 (0, 1, 2, 3)
        outcome: 结果
        X_eval: 评估特征
        
    Returns:
        pred_mus: (n_eval, num_treatments) - 每种处理的预测结果
        pred_cate: (n_eval, num_treatments - 1) - 相对 T=0 的 CATE
    """
    # 实现 T-Learner
    num_treatments = 4
    n_eval = len(X_eval)
    pred_mus = np.zeros((n_eval, num_treatments))
    
    # 为每种处理筛选样本并训练模型
    for k in range(num_treatments):
        # 1. 为每种处理筛选样本: mask = (treatment == k)
        mask = (treatment == k)
        
        # 2. 训练模型: model.fit(X[mask], outcome[mask])
        model = GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)
        model.fit(X[mask], outcome[mask])
        
        # 3. 预测: pred_mus[:, k] = model.predict(X_eval)
        pred_mus[:, k] = model.predict(X_eval)
    
    # 计算 CATE（相对于 T=0）
    pred_cate = pred_mus[:, 1:] - pred_mus[:, 0:1]
    
    return pred_mus, pred_cate

print("✅ multi_treatment_t_learner 函数定义完成（已完成 TODO 部分）")

### 📝 参考答案

In [ ]:
# ============================================================
# 参考答案：Multi-Treatment T-Learner
# ============================================================

def multi_treatment_t_learner(X, treatment, outcome, X_eval):
    """
    Multi-Treatment T-Learner - 完整实现
    """
    num_treatments = 4
    n_eval = len(X_eval)
    pred_mus = np.zeros((n_eval, num_treatments))
    
    for k in range(num_treatments):
        mask = treatment == k
        model = GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)
        model.fit(X[mask], outcome[mask])
        pred_mus[:, k] = model.predict(X_eval)
    
    pred_cate = pred_mus[:, 1:] - pred_mus[:, 0:1]
    
    return pred_mus, pred_cate

# 运行 T-Learner
print("训练 T-Learner...")
t_learner_mus, t_learner_cate = multi_treatment_t_learner(
    data['X'], data['treatment'], data['outcome'], data['X']
)
print("✅ T-Learner 训练完成")

# 对比
print("\n" + "=" * 60)
print("方法对比: DragonNet vs T-Learner")
print("=" * 60)

treatment_names = ['5折券', '7折券', '满减券']

print(f"\n{'方法':<15} {'处理':<10} {'RMSE':<10} {'相关系数':<10}")
print("-" * 45)

for i, name in enumerate(treatment_names):
    # DragonNet
    rmse_dn = np.sqrt(np.mean((pred_cate[:, i] - data['true_cate'][:, i]) ** 2))
    corr_dn = np.corrcoef(pred_cate[:, i], data['true_cate'][:, i])[0, 1]
    
    # T-Learner
    rmse_tl = np.sqrt(np.mean((t_learner_cate[:, i] - data['true_cate'][:, i]) ** 2))
    corr_tl = np.corrcoef(t_learner_cate[:, i], data['true_cate'][:, i])[0, 1]
    
    print(f"{'DragonNet':<15} {name:<10} {rmse_dn:<10.3f} {corr_dn:<10.3f}")
    print(f"{'T-Learner':<15} {name:<10} {rmse_tl:<10.3f} {corr_tl:<10.3f}")
    print("-" * 45)

---

## Part 11: 业务应用：智能发券策略

现在我们将模型应用到实际业务场景中。

In [ ]:
def generate_coupon_strategy_report(model, scaler, data):
    """
    生成智能发券策略报告
    """
    model.eval()
    X_scaled = torch.FloatTensor(scaler.transform(data['X']))
    
    with torch.no_grad():
        propensity, pred_mus = model(X_scaled)
        pred_cate = model.predict_cate(X_scaled, reference_treatment=0)
        
    pred_mus = pred_mus.numpy()
    pred_cate = pred_cate.numpy()
    
    # 最优策略选择
    optimal_treatment = np.argmax(pred_mus, axis=1)
    
    # 计算策略收益（考虑成本）
    # 假设成本：5折券成本5元，7折券成本3元，满减券成本2元
    costs = np.array([0, 5, 3, 2])
    net_values = pred_mus - costs
    optimal_with_cost = np.argmax(net_values, axis=1)
    
    print("=" * 70)
    print("智能发券策略报告")
    print("=" * 70)
    
    print("\n1. 策略分布（不考虑成本）:")
    for i, name in enumerate(['不发券', '5折券', '7折券', '满减券']):
        count = np.sum(optimal_treatment == i)
        pct = count / len(optimal_treatment) * 100
        print(f"   {name}: {count:5d} ({pct:.1f}%)")
    
    print("\n2. 策略分布（考虑成本）:")
    for i, name in enumerate(['不发券', '5折券', '7折券', '满减券']):
        count = np.sum(optimal_with_cost == i)
        pct = count / len(optimal_with_cost) * 100
        print(f"   {name}: {count:5d} ({pct:.1f}%)")
    
    print("\n3. 用户群体分析:")
    # 按价格敏感度分组
    price_sens = data['X'][:, 1]
    low_sens = price_sens < np.percentile(price_sens, 33)
    high_sens = price_sens > np.percentile(price_sens, 67)
    
    print("\n   低价格敏感度用户:")
    for i, name in enumerate(['不发券', '5折券', '7折券', '满减券']):
        pct = np.sum(optimal_with_cost[low_sens] == i) / np.sum(low_sens) * 100
        print(f"      {name}: {pct:.1f}%")
    
    print("\n   高价格敏感度用户:")
    for i, name in enumerate(['不发券', '5折券', '7折券', '满减券']):
        pct = np.sum(optimal_with_cost[high_sens] == i) / np.sum(high_sens) * 100
        print(f"      {name}: {pct:.1f}%")
    
    print("\n4. 预期效果（相比随机发券）:")
    value_model = np.mean([net_values[i, optimal_with_cost[i]] for i in range(len(net_values))])
    value_random = np.mean([net_values[i, np.random.randint(4)] for i in range(len(net_values))])
    print(f"   模型推荐: {value_model:.2f}")
    print(f"   随机发券: {value_random:.2f}")
    print(f"   提升: {(value_model - value_random):.2f} ({(value_model - value_random) / abs(value_random) * 100:.1f}%)")
    
    return optimal_treatment, optimal_with_cost

optimal_treatment, optimal_with_cost = generate_coupon_strategy_report(model, scaler, data)

In [ ]:
# 可视化策略分布与用户特征的关系
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 策略与购买力的关系
ax1 = axes[0]
for i, (name, color) in enumerate(zip(['不发券', '5折券', '7折券', '满减券'], 
                                      ['#95a5a6', '#e74c3c', '#3498db', '#2ecc71'])):
    mask = optimal_with_cost == i
    ax1.scatter(data['X'][mask, 0], data['X'][mask, 1], 
               alpha=0.4, s=10, c=color, label=name)
ax1.set_xlabel('购买力')
ax1.set_ylabel('价格敏感度')
ax1.set_title('最优发券策略分布')
ax1.legend()

# CATE 热力图 - 5折券
ax2 = axes[1]
# 创建网格
x_range = np.linspace(data['X'][:, 0].min(), data['X'][:, 0].max(), 50)
y_range = np.linspace(data['X'][:, 1].min(), data['X'][:, 1].max(), 50)
xx, yy = np.meshgrid(x_range, y_range)

# 生成预测输入
grid_X = np.zeros((len(x_range) * len(y_range), 10))
grid_X[:, 0] = xx.ravel()
grid_X[:, 1] = yy.ravel()

# 预测 CATE
model.eval()
grid_X_scaled = torch.FloatTensor(scaler.transform(grid_X))
with torch.no_grad():
    grid_cate = model.predict_cate(grid_X_scaled, reference_treatment=0)
grid_cate = grid_cate[:, 0].numpy().reshape(xx.shape)  # 5折券的 CATE

im = ax2.contourf(xx, yy, grid_cate, levels=20, cmap='RdYlGn')
plt.colorbar(im, ax=ax2, label='CATE (5折券 vs 不发券)')
ax2.set_xlabel('购买力')
ax2.set_ylabel('价格敏感度')
ax2.set_title('5折券 CATE 热力图')

plt.tight_layout()
plt.show()

print("\n观察：")
print("1. 高价格敏感度用户对折扣券响应更强")
print("2. 不同用户群体最优策略不同，体现了精细化运营的价值")
print("3. CATE 热力图展示了处理效果的异质性")

---

## 💡 思考题

1. **架构设计**：如果处理数量很多（如 100 种），如何改进网络架构？
   - 提示：考虑处理 Embedding、参数共享

2. **Loss 设计**：如果某些处理的样本量极少（如 1%），如何修改 Loss？
   - 提示：参考 Deep Dive 01 的 Focal Loss 思路

3. **业务约束**：如果有预算约束（如最多给 30% 用户发券），如何修改策略？
   - 提示：考虑带约束的优化问题

4. **因果识别**：如果存在未观测混淆变量，DragonNet 的估计会有什么问题？
   - 提示：无可忽略性假设

---

## 📝 总结

本 notebook 我们学习了：

1. **DragonNet 原理**：共享表示层 + 倾向得分头 + 结果预测头

2. **多处理扩展**：
   - 倾向得分：二分类 → 多分类
   - 结果预测：2 个头 → K 个头
   - Targeted Reg：泛化到多处理

3. **实现细节**：
   - 使用 `nn.ModuleList` 管理多个头
   - 使用 `gather` 选择观测处理的预测
   - 数值稳定性处理（eps、clamp）

4. **业务应用**：
   - 最优处理选择
   - 考虑成本的策略优化
   - 用户群体分析

### 面试要点

- 能解释 DragonNet 的设计动机和架构
- 能手写多处理扩展的核心代码
- 理解 Targeted Regularization 的作用
- 能讨论实际应用中的挑战和解决方案

---

## 🔗 参考资料

1. Shi, C., Blei, D. M., & Veitch, V. (2019). Adapting Neural Networks for the Estimation of Treatment Effects. NeurIPS.
2. Johansson, F. D., Shalit, U., & Sontag, D. (2016). Learning Representations for Counterfactual Inference. ICML.
3. Kunzel, S. R., et al. (2019). Metalearners for Estimating Heterogeneous Treatment Effects using Machine Learning. PNAS.